In [1]:
import pandas as pd
import warnings
from openpyxl import load_workbook

with warnings.catch_warnings(record=True):
        warnings.simplefilter("always")
                              
        #Majority of info comes from Active Employee Report
        active = pd.read_excel('RPT00035_-_Active_Employee_Details_2024_04_19.xlsx', engine='openpyxl')

        #Info has the company service date:
        info = pd.read_excel('RPT00066 - Employee Information_2024_04_19.xlsx')

        #Email Listings has the contact info:
        email = pd.read_excel('RPT00324_-_Email_Listings_2024_04_19.xlsx', engine='openpyxl')

In [2]:
#Getting the join key and email address info:
email = email[['Employee ID', 'Email Address']]

#Getting only the email:
email['Email Address'] = email['Email Address'].str.split("(").str[0].str.strip()

#email

In [3]:
#Getting info out of active:
active = active[['Worker',
                 'Last Name',
                 'First Name',
                 'Employee ID',
                 'Supervisory Organization',  #Extract Department and Supervisor
                 'Business Title',
                 'Employee Type',
                 'Supervisory Level',
                 'Birthdate',                 #Remove after you get calculation of Generation
                 'Age',
                 'Ethnicity',                 #Get Ethnicity before " ("
                 'Gender',
                 'Annual Rate',
                 'Location',
                 'FTE'                       #Create 'Time Type'
                ]]

# Extracting department information from 'Supervisory Organization' column
active['Department'] = active['Supervisory Organization'].str.extract(r'340\s(.*?)\s\(')

# Extracting supervisor information from 'Supervisory Organization' column
active['Supervisor'] = active['Supervisory Organization'].apply(lambda x: x[x.find("(")+1:x.find(")")])

#Removing Inherited from the Supervisor column:
active['Supervisor'] = active['Supervisor'].str.split("(").str[0].str.strip()
                                                  
#Central Office

# Apply the function to create the new column 'CO or District'


# Define a lambda function to apply the conditions
active['CO or District'] = active.apply(lambda row: 'District' if (isinstance(row['Department'], str) and 'District' in row['Department']) or 
                                                              (isinstance(row['Business Title'], str) and 'Regional Administrative Director' in row['Business Title']) 
                                                     else 'Central Office', axis=1)
    
# Convert 'Birthdate' column to datetime format and then format it to 'mm/dd/yyyy'
active['Birthdate'] = pd.to_datetime(active['Birthdate']).dt.strftime('%m/%d/%Y')

# Create a new column 'generation' based on birth year
active['Birthyear'] = pd.to_datetime(active['Birthdate']).dt.year
active['Generation'] = pd.cut(active['Birthyear'],
                              bins=[1910, 1927, 1945, 1964, 1980, 1996, pd.to_datetime('now').year],
                              labels=['Greatest Generation', 'Silent Generation', 'Baby Boomer', 'Generation X', 'Millennial', 'Generation Z'])


#Removing Inherited from the Supervisor column:
active['Ethnicity'] = active['Ethnicity'].str.split("(").str[0].str.strip()

# Reordering the columns while keeping 'Supervisory Organization' in its original position
columns_order = list(active.columns)
columns_order.insert(columns_order.index('Employee ID') + 1, 'CO or District')
columns_order.insert(columns_order.index('Supervisory Organization') + 1, 'Department')
columns_order.insert(columns_order.index('Department') + 1, 'Supervisor')
columns_order.insert(columns_order.index('Age') + 1, 'Generation')
active = active.reindex(columns=columns_order)

# Dropping the 'Birthyear' column if not needed
active.drop(columns=['Birthyear', 'Birthdate'], inplace=True)
#Dropping the repeat columns:
active = active.iloc[:, :-4]

# Define a function to determine the Time Type based on FTE value
def determine_time_type(fte):
    if fte == 1:
        return 'Full-time'
    else:
        return 'Part-time'

# Apply the function to create the 'Time Type' column
active['Time Type'] = active['FTE'].apply(determine_time_type)

#Removing Oklahoma State Athletic Commission, since it is not under Keith Reed. 
active = active[active['Department'] != 'Okla State Athletic Commission']

#Removing contract workers:
active = active[active['Employee Type'].notna()]

#Removing OSAC Administrator, since they are not under Keith Reed:
active = active[active['Business Title'] != 'OSAC Administrator']

#active

C:\Users\ChaseC\AppData\Local\Temp\ipykernel_25080\2340972581.py:44: FutureWarning: The parsing of 'now' in pd.to_datetime without `utc=True` is deprecated. In a future version, this will match Timestamp('now') and Timestamp.now()
  bins=[1910, 1927, 1945, 1964, 1980, 1996, pd.to_datetime('now').year],


In [4]:
#Getting the join key and email address info:
info = info[['Employee ID', 'Company Service Date']]

# Convert 'Company Service Date' to datetime format
info['Company Service Date'] = pd.to_datetime(info['Company Service Date'])

# Calculate the difference between 'Company Service Date' and today's date to get service years
info['Company Service Years'] = (pd.Timestamp.now() - info['Company Service Date']).dt.days / 365

# Create 'FY2018 RIF' column based on the condition
info['FY2018 RIF'] = info['Company Service Date'].apply(
    lambda date: 'Endured RIF' if date < pd.Timestamp('2018-04-01') else 'Post RIF')

info['Pandemic'] = info['Company Service Date'].apply(
    lambda date: 'Endured RIF' if date < pd.Timestamp('2018-04-01') else 'Post RIF')

# Function to determine the pandemic phase
def determine_pandemic_phase(date):
    if date < pd.Timestamp('2020-03-14'):
        return "Pre-pandemic (Before March 14th, 2020)"
    elif date < pd.Timestamp('2021-03-14'):
        return "1st Year of Pandemic (March 14th, 2020 - March 13th, 2021)"
    elif date < pd.Timestamp('2022-03-14'):
        return "2nd Year of Pandemic (March 14th, 2021 - March 13th, 2022)"
    elif date < pd.Timestamp('2023-04-11'):
        return "3rd Year of Pandemic (March 14th, 2022 - April 11th, 2023)"
    else:
        return "Post Pandemic (April 11th, 2023 - Present)"

# Create 'Pandemic' column
info['Pandemic'] = info['Company Service Date'].apply(determine_pandemic_phase)
#info

In [5]:
#Fixing Keith Reed's Info:
# Assuming final_df is your DataFrame
active.loc[active['Worker'] == 'Keith Reed', 'Department'] = 'Secretary of Health and Mental Health'

In [6]:
# Perform left join on 'active' and 'info' DataFrames
final_df = pd.merge(active, info, on='Employee ID', how='left')

# Perform left join on 'merged_df' and 'email' DataFrame
final_df = pd.merge(final_df, email, on='Employee ID', how='left')

#final_df

In [9]:
writer = pd.ExcelWriter('OSDH Employee Info.xlsx', engine='xlsxwriter')
final_df.to_excel(writer, sheet_name='OSDH Employee Info', index=False) 

#Saves Dataframe with filters turned on:
worksheet = writer.sheets['OSDH Employee Info']
worksheet.autofilter(0,0,final_df.shape[0],final_df.shape[1]-1) 

    #Adjusts sizes of columns for easier reading:
for column in final_df:
    column_length = max(final_df[column].astype(str).map(len).max(), len(column))
    col_idx = final_df.columns.get_loc(column)
    writer.sheets['OSDH Employee Info'].set_column(col_idx, col_idx, column_length)

writer.close()

rows_with_blank_cell = final_df[final_df.isna().any(axis=1)]
pd.set_option('display.max_columns', None)
rows_with_blank_cell[["Worker", "Company Service Date", "Company Service Years"]]